# 37.3 Final Capstone Prep
---


<br></br>
This notebook contains the code used to scrape and clean the data for the whisky database

In [817]:
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
import warnings
from warnings import warn
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import itertools
import regex as re

warnings.filterwarnings("ignore")

In [330]:
# HTML and category dictionary as it is being scraped by category per the Whisky Advocate dropdown
html1 = 'https://www.whiskyadvocate.com/ratings-reviews/?search=&submit=+&brand_id=0&rating=0&price=0&category=' 
html2 = '&styles_id=0&issue_id=0'
category_specifier = {'Scotch':'1%2C3%2C4%2C6%2C51', 
                     'Irish':'8%2C17%2C20%2C24%2C37%2C54%2C56%2C65',
                     'American':'5%2C7%2C15%2C18%2C33%2C34%2C40%2C42%2C73',
                     'Canadian':'10',
                     'Japanese':'11',
                     'World':'9%2C11%2C12%2C13%2C16%2C19%2C21%2C23%2C26%2C27%2C28%2C29%2C30%2C31%2C32%2C39%2C44%2C46%2C47%2C48%2C49%2C50%2C52%2C53%2C55%2C57%2C58%2C59%2C60%2C61%2C66%2C70%2C71%2C72',
                     'Flavored':'41',
                     'Other':'2%2C45%2C46',
                     # 'Bourbon/Tennessee' not included as it is part of the 'American' category
                     'Rye':'5',
                     'Single/Blended Malt':'1%2C2%2C3%2C16%2C18%2C21%2C23%2C59%2C61',
                     'Single/Blended Grain':'6%2C20%2C51%2C58',
                     'Blended':'3%2C4%2C17%2C42%2C47%2C54%2C55%2C57'
                    }

In [218]:
# Testing the viability of the html tags
response = requests.get(str(html1 + category_specifier['Rye'] + html2))

if (response.ok):
    data = response.text

soup = BeautifulSoup(data, 'html.parser')

response.ok

True

In [224]:
# Empty lists to build
points = []
labels = []
categories = []
prices = []
reviews = []
reviewers = []
years = []

In [239]:
# A list of the lists
scraped_lists = [points, labels, categories, prices, reviews, reviewers, years]

In [226]:
# Getting the scrape points
def process_data(soup):
    points.append([point.get_text() for point in soup.select('.review-top > h2 > span')])
    labels.append([label.get_text() for label in soup.select('.printable-section > h1')])
    categories.append([category.get_text() for category in soup.select('.entry-meta > span > span:nth-of-type(1)')])
    prices.append([price.get_text() for price in soup.select('.entry-meta > span > span:nth-of-type(3)')])
    reviews.append([review.get_text() for review in soup.select('.printable-section > div:nth-of-type(1) > p')])
    reviewers.append([reviewer.get_text() for reviewer in soup.select('.printable-section > div:nth-of-type(2) > p > span')])
    # Gets the last four characters, which is the year
    years.append([year.get_text()[-4:] for year in soup.select('.printable-section > div:nth-of-type(2) > p > a')])

In [227]:
def flatten_data(scraped_dict):
    for i, j in enumerate(scraped_lists):
        collapsed = list(itertools.chain.from_iterable(j))
        scraped_lists[i] = collapsed

In [228]:
# Building the scraper
start = time()

headers = {'user-agent':'whisky info scraper - educational project (dancassinatwork@gmail.com)'}

for i in category_specifier:
    clear_output(wait=True)

    response = requests.get(str(html1 + category_specifier[i] + html2), headers=headers)

    if (response.ok):
        data = response.text
        soup = BeautifulSoup(data, 'html.parser')
        process_data(soup)

    sleep(randint(1,5))

    elapsed_time = time() - start
    print(f'Categories: {len(labels)}, Frequency: {len(labels)/elapsed_time} requests/s')



print('Scraping Complete')

Whiskies: 11, Frequency: 0.1272593429836425 requests/s
Scraping Complete


In [ ]:
flatten_data(scraped_lists)

In [248]:
len(scraped_lists[1])

8250

In [837]:
initial_df = pd.DataFrame(scraped_lists)
whisky = initial_df.T

In [838]:
# Apparently some categories in the dropdown had overlapping whiskies
whisky.duplicated(subset=4).sum()

3174

In [839]:
whisky.drop_duplicates(subset=4,inplace=True)

In [840]:
whisky.reset_index(drop=True, inplace=True)

In [841]:
whisky.rename(columns={0:'rating', 1:'label', 2:'category', 3:'price', 
                       4:'review', 5:'reviewer', 6:'year'}, inplace=True)


In [842]:
whisky.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076 entries, 0 to 5075
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rating    5076 non-null   object
 1   label     5076 non-null   object
 2   category  5076 non-null   object
 3   price     5076 non-null   object
 4   review    5076 non-null   object
 5   reviewer  5076 non-null   object
 6   year      5076 non-null   object
dtypes: object(7)
memory usage: 277.7+ KB


In [843]:
whisky.tail()

,rating,label,category,price,review,reviewer,year
5071,80,"The Hilhaven Lodge, 40%",Miscellaneous,50,Straw to light tawny. This blend of straight A...,Fred Minnick,2017
5072,79,"Orphan Barrel Whoop & Holler, 42%",Miscellaneous,175,This composite includes 28 year old barrels th...,Fred Minnick,2017
5073,76,Buffalo Trace Experimental Collection Organic ...,Miscellaneous,46,"Youth and rawness offer the first impression, ...",Lew Bryson,2015
5074,75,"Rebel Yell American Whiskey, 45%",Miscellaneous,30,"A blend of bourbon and rye whiskeys, 2 years o...",Fred Minnick,2017
5075,73,"Virginia Black American Whiskey, 40%",Miscellaneous,35,"Sourced whiskey from MGP, it’s fairly muted wi...",John Hansell,2009


# REGEX
---
<br></br>
Using regex to clean columns and scrape values from one column to create new columns

In [900]:
# Remove ' ABV' from end of label
whisky['label'] = whisky.label.replace(to_replace=['[A][B][V]'], value='', regex=True).str.strip()
# Remove ',' from end of label
whisky['label'] = whisky.label.replace('\,$', '', regex=True)
# Removing characters from price column and replacing with space or no space depending on character
whisky['price'] = whisky.price.replace(['\.', '\/', '\$'],' ', regex=True)
whisky['price'] = whisky.price.replace('\,','', regex=True)

In [901]:
# Using regex to create a new column for the ABV
whisky['abv'] = whisky.label.str.extract(r'([0-9][0-9]\.[0-9][0-9]%$|[0-9][0-9]%$|[0-9][0-9]\.[0-9]%$)')
# Remove abv percents
whisky['abv'] = whisky.abv.replace('%','', regex=True)
# ABV to float
whisky['abv'] = pd.to_numeric(whisky['abv'], downcast="float")

In [902]:
# Future update remove [$€£] from reviews and the numbers that follow them

In [903]:
pd.options.display.float_format = '{:,.2f}'.format

In [904]:
whisky.head()

,rating,label,category,price,review,reviewer,year,abv,price_float,vintage,age
0,97,"Black Bowmore 42 year old 1964 vintage, 40.5%",Single Malt Scotch,4500,What impresses me most is how this whisky evol...,John Hansell,2008,40.50,"4,500.00",1964,42
1,97,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,13500,There have been some legendary Bowmores from t...,Dave Broom,2012,42.90,"13,500.00",1964,46
2,97,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,225,"Magnificently powerful and intense. Caramels, ...",Jonny McCormick,2018,40.00,225.00,NaN,NaN
3,96,"Glenlivet Cellar Collection 1969 vintage, 50.8%",Single Malt Scotch,750,It’s great that Glenlivet releases whiskies un...,John Hansell,2007,50.80,750.00,1969,NaN
4,96,The Macallan 29 year old 1976 Vintage (Cask #1...,Single Malt Scotch,1500,Classic sherry cask-aged Macallan. Antique amb...,John Hansell,2008,45.40,"1,500.00",1976,29


In [905]:
# Creating a new price_float column and turning it into float dtype
whisky['price_float'] = whisky.price.str.split('\s',1).str[0].str.strip()
whisky['price_float'] = pd.to_numeric(whisky['price_float'], downcast="float")

In [906]:
# extracting the age of the whisky by finding " year" and getting the preceding numbers
#whisky['age'] = whisky.label.str.extract(r'(\d+)(?=( year)|( Year))')
whisky['age'] = whisky.label.str.extract('(\d+(?= year| Year))', expand=True)

In [907]:
# extracting the vintage of the whisky
whisky['vintage'] = whisky.label.str.extract(r'([2][0][01][0-9]|[1][9][4-9][0-9])')

In [908]:
# Only one price value wasn't extracted via regex. Easiest to fix with simple assignment
whisky.price_float[5016] = 130
# Only two years and reviewers weren't scraped. Double checked with website and hardcoded
#whisky.year[6936], whisky.year[6937] = '2018', '2016'
#whisky.reviewer[6936], whisky.reviewer[6937] = 'Fred Minnick', 'Fred Minnick'

In [909]:
whisky.head()

,rating,label,category,price,review,reviewer,year,abv,price_float,vintage,age
0,97,"Black Bowmore 42 year old 1964 vintage, 40.5%",Single Malt Scotch,4500,What impresses me most is how this whisky evol...,John Hansell,2008,40.50,"4,500.00",1964,42
1,97,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,13500,There have been some legendary Bowmores from t...,Dave Broom,2012,42.90,"13,500.00",1964,46
2,97,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,225,"Magnificently powerful and intense. Caramels, ...",Jonny McCormick,2018,40.00,225.00,NaN,NaN
3,96,"Glenlivet Cellar Collection 1969 vintage, 50.8%",Single Malt Scotch,750,It’s great that Glenlivet releases whiskies un...,John Hansell,2007,50.80,750.00,1969,NaN
4,96,The Macallan 29 year old 1976 Vintage (Cask #1...,Single Malt Scotch,1500,Classic sherry cask-aged Macallan. Antique amb...,John Hansell,2008,45.40,"1,500.00",1976,29


In [910]:
whisky.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076 entries, 0 to 5075
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rating       5076 non-null   object 
 1   label        5076 non-null   object 
 2   category     5076 non-null   object 
 3   price        5076 non-null   object 
 4   review       5076 non-null   object 
 5   reviewer     5076 non-null   object 
 6   year         5076 non-null   object 
 7   abv          5050 non-null   float32
 8   price_float  5076 non-null   float32
 9   vintage      1147 non-null   object 
 10  age          1908 non-null   object 
dtypes: float32(2), object(9)
memory usage: 396.7+ KB


In [912]:
# Inspecting rows with NaN values to see if anything was missed by regex
whisky[whisky.isna().any(axis=1)]

,rating,label,category,price,review,reviewer,year,abv,price_float,vintage,age
2,97,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,225,"Magnificently powerful and intense. Caramels, ...",Jonny McCormick,2018,40.00,225.00,NaN,NaN
3,96,"Glenlivet Cellar Collection 1969 vintage, 50.8%",Single Malt Scotch,750,It’s great that Glenlivet releases whiskies un...,John Hansell,2007,50.80,750.00,1969,NaN
5,96,"The Dalmore 50 year old, 52.8%",Single Malt Scotch,1500,The Dalmore is one of a handful of whiskies th...,John Hansell,2009,52.80,"1,500.00",NaN,50
6,96,"Gold Bowmore 1964 vintage, 42.4%",Single Malt Scotch,6250,Deep gold color. Surprisingly lively on the no...,John Hansell,2009,42.40,"6,250.00",1964,NaN
7,96,"Bowmore 40 year old, 44.8%",Single Malt Scotch,11000,"Definitely showing its age, but not in a bad w...",John Hansell,2011,44.80,"11,000.00",NaN,40
...,...,...,...,...,...,...,...,...,...,...,...
5071,80,"The Hilhaven Lodge, 40%",Miscellaneous,50,Straw to light tawny. This blend of straight A...,Fred Minnick,2017,40.00,50.00,NaN,NaN
5072,79,"Orphan Barrel Whoop & Holler, 42%",Miscellaneous,175,This composite includes 28 year old barrels th...,Fred Minnick,2017,42.00,175.00,NaN,NaN
5073,76,Buffalo Trace Experimental Collection Organic ...,Miscellaneous,46,"Youth and rawness offer the first impression, ...",Lew Bryson,2015,45.00,46.00,NaN,NaN
5074,75,"Rebel Yell American Whiskey, 45%",Miscellaneous,30,"A blend of bourbon and rye whiskeys, 2 years o...",Fred Minnick,2017,45.00,30.00,NaN,NaN


In [913]:
whisky.head()

,rating,label,category,price,review,reviewer,year,abv,price_float,vintage,age
0,97,"Black Bowmore 42 year old 1964 vintage, 40.5%",Single Malt Scotch,4500,What impresses me most is how this whisky evol...,John Hansell,2008,40.50,"4,500.00",1964,42
1,97,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,13500,There have been some legendary Bowmores from t...,Dave Broom,2012,42.90,"13,500.00",1964,46
2,97,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,225,"Magnificently powerful and intense. Caramels, ...",Jonny McCormick,2018,40.00,225.00,NaN,NaN
3,96,"Glenlivet Cellar Collection 1969 vintage, 50.8%",Single Malt Scotch,750,It’s great that Glenlivet releases whiskies un...,John Hansell,2007,50.80,750.00,1969,NaN
4,96,The Macallan 29 year old 1976 Vintage (Cask #1...,Single Malt Scotch,1500,Classic sherry cask-aged Macallan. Antique amb...,John Hansell,2008,45.40,"1,500.00",1976,29


In [914]:
final_df = whisky[['rating', 'label', 'category', 'review', 'reviewer', 'year',
       'abv', 'price_float', 'age', 'vintage']]

final_df.rename(columns={'price_float':'price'}, inplace=True)

final_df.head()

,rating,label,category,review,reviewer,year,abv,price,age,vintage
0,97,"Black Bowmore 42 year old 1964 vintage, 40.5%",Single Malt Scotch,What impresses me most is how this whisky evol...,John Hansell,2008,40.50,"4,500.00",42,1964
1,97,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,There have been some legendary Bowmores from t...,Dave Broom,2012,42.90,"13,500.00",46,1964
2,97,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,"Magnificently powerful and intense. Caramels, ...",Jonny McCormick,2018,40.00,225.00,NaN,NaN
3,96,"Glenlivet Cellar Collection 1969 vintage, 50.8%",Single Malt Scotch,It’s great that Glenlivet releases whiskies un...,John Hansell,2007,50.80,750.00,NaN,1969
4,96,The Macallan 29 year old 1976 Vintage (Cask #1...,Single Malt Scotch,Classic sherry cask-aged Macallan. Antique amb...,John Hansell,2008,45.40,"1,500.00",29,1976


In [915]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5076 entries, 0 to 5075
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   rating    5076 non-null   object 
 1   label     5076 non-null   object 
 2   category  5076 non-null   object 
 3   review    5076 non-null   object 
 4   reviewer  5076 non-null   object 
 5   year      5076 non-null   object 
 6   abv       5050 non-null   float32
 7   price     5076 non-null   float32
 8   age       1908 non-null   object 
 9   vintage   1147 non-null   object 
dtypes: float32(2), object(8)
memory usage: 357.0+ KB


In [916]:
final_df.to_csv('whisky1.csv')